# DROWSINESS DETECTION USING OPENCV AND DLIB 

####   PROJECT BY SURUCHI KUMARI

In [1]:
pip install scipy


[notice] A new release of pip available: 22.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip --version

pip 22.2 from c:\Users\SURUCHI\AppData\Local\Programs\Python\Python310\lib\site-packages\pip (python 3.10)

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install --upgrade pip

  Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)

[notice] A new release of pip available: 22.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: To modify pip, please run the following command:
C:\Users\SURUCHI\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [4]:
pip install opencv-python


[notice] A new release of pip available: 22.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install cmake


[notice] A new release of pip available: 22.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install wheel


[notice] A new release of pip available: 22.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install dlib


[notice] A new release of pip available: 22.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# import the necessary packages
from scipy.spatial import distance as dist
from threading import Thread
import numpy as np
import cv2
import time
import dlib
#import playsound
detector =  dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [9]:
import numpy as np

In [10]:
def fetching_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    
    
    
    if len(rects)>1:
        return"error"
    if len(rects)== 0:
        return"error"
    return np.matrix([[p.x, p.y] for p in predictor(gray, rects[0]).parts()])

In [11]:
def annotate_landmarks(image, landmarks):
    image =image.copy()
    for idx,point in enumerate(landmarks):
        pos=(point[0,0],point[0,1])
        cv2.putText(image,str(idx),pos,fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,fontScale=0.4,color=(255,255,0))
        cv2.circle(image,pos,3,color=(0,0,255))
    return image    

def top_lip(landmarks):
    top_lip_pts = []
    for i in range(50,53):
        top_lip_pts.append(landmarks[i])
    for i in range(61,64):
        top_lip_pts.append(landmarks[i])
    top_lip_mean = np.mean(top_lip_pts, axis=0)
    return int(top_lip_mean[:,1])

def bottom_lip(landmarks):
    bottom_lip_pts = []
    for i in range(65,68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56,59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_mean = np.mean(bottom_lip_pts, axis=0)
    return int(bottom_lip_mean[:,1])



In [12]:
def eye_aspect_ratio(eye_pts):

    A = dist.euclidean(eye_pts[1], eye_pts[5])
    B = dist.euclidean(eye_pts[2], eye_pts[4])
    C = dist.euclidean(eye_pts[0], eye_pts[3])

    ear = (A + B) / (2.0 * C)
    return ear

def left_eye_landmark(landmarks):
    left_eye_pts = []
    for i in range(36,42):
        left_eye_pts.append(landmarks[i])
    return(eye_aspect_ratio(left_eye_pts))

def right_eye_landmark(landmarks):
    right_eye_pts = []
    for i in range(42,48):
        right_eye_pts.append(landmarks[i])
    return(eye_aspect_ratio(right_eye_pts))
    

In [13]:
def drowsiness_detect(image):
    landmarks= fetching_landmarks(image)
    
    
    if landmarks =="error":
        return 0,0,image
    
    image_with_landmarks= annotate_landmarks(image, landmarks)
    top_lip_center=top_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(top_lip_center - bottom_lip_center)
    leftEAR = left_eye_landmark(landmarks)
    rightEAR = right_eye_landmark(landmarks)
    ear = (leftEAR + rightEAR) / 2.0
    return ear, lip_distance, image_with_landmarks

In [14]:
## THRESHOLD VALUES
EYE_THRESH = 0.3
EYE_CONSEC_FRAMES = 20

COUNTER_4_YAWN = 0
COUNTER_4_EYE = 0
#ALARM_ON = False

LIP_THRESH = 24
LIP_CONSEC_FRAME = 5


In [16]:

vs = cv2.VideoCapture(0)

while True:
    check,frame = vs.read()
    ear, lip_distance, image_with_landmarks = drowsiness_detect(frame)
    if lip_distance > LIP_THRESH:
        COUNTER_4_YAWN += 1
        if COUNTER_4_YAWN >= LIP_CONSEC_FRAME:
            cv2.putText(frame," FEELING SLEEPY   TAKE REST ", (50,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
    else:
        COUNTER_4_YAWN = 0
        
    if ear < EYE_THRESH:
        COUNTER_4_EYE += 1
        if COUNTER_4_EYE >= EYE_CONSEC_FRAMES:
            cv2.putText(frame, "Drowsiness ALERT!", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    else:
        COUNTER_4_EYE = 0
            
    cv2.putText(frame, "EAR: {:.2f}".format(ear), (460, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
 
    cv2.imshow("Frame", frame)
    cv2.imshow("Landmarks", image_with_landmarks)
    key = cv2.waitKey(1) & 0xFF
 

    if key == ord("q"):
        break


vs.release()
cv2.destroyAllWindows()



C:\Users\SURUCHI\AppData\Local\Temp\ipykernel_16168\2431880504.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if landmarks =="error":


KeyboardInterrupt: 